In [ ]:
# Install qubex library if not installed
# !pip install git+https://github.com/amachino/qubex.git

In [ ]:
import numpy as np
import qubex as qx

In [ ]:
# Create a flat-top pulse
flattop = qx.pulse.FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)

# Plot the pulse
flattop.plot()

# Print the pulse values
print(flattop.values)

In [ ]:
# Scale the amplitude of the pulse
flattop.scaled(1.5).plot()

# Shift the phase of the pulse
flattop.shifted(np.pi).plot()

# Repeat the pulse
flattop.repeated(3).plot()

In [ ]:
# Create a Gaussian pulse
gaussian = qx.pulse.Gaussian(
    duration=60,
    amplitude=0.5,
    sigma=10,
)

# Create a pulse array
arr = qx.PulseArray(
    [
        gaussian.shifted(np.pi / 2),
        qx.pulse.VirtualZ(np.pi / 4),
        gaussian.shifted(-np.pi / 2),
    ]
)

# Plot the pulse array
arr.plot()

# Print the pulse array values
print(arr.values)

In [ ]:
# Create an arbitrary pulse
iq_array = [
    0.1 + 0.2j,
    0.2 + 0.3j,
    0.3 + 0.4j,
    0.4 + 0.6j,
    0.5 + 0.6j,
]
arbit = qx.Pulse(iq_array)

# Plot the pulse
arbit.plot()

# Plot the reverse of the pulse
arbit.reversed().plot()

In [ ]:
# Define channel labels
labels = ["Q00", "Q01", "Q02"]

# Create a pulse schedule for the channels
with qx.PulseSchedule(labels) as seq1:
    seq1.add("Q00", flattop.repeated(5))
    seq1.add("Q01", arr.scaled(2))
    seq1.barrier()
    seq1.add("Q00", arbit.repeated(3))
    seq1.barrier(["Q00", "Q01"])
    seq1.add("Q01", qx.pulse.FlatTop(duration=100, amplitude=1, tau=10))
    seq1.add("Q02", qx.PulseArray([gaussian, qx.pulse.VirtualZ(np.pi / 2), gaussian]))

# Plot the pulse schedule
seq1.plot()

# Get the PulseArray dictionary of the channels
seq1.get_sequences()

In [ ]:
# Get the sampled sequences of the channels
seq1.get_sampled_sequences()

In [ ]:
# Create a pulse schedule using the previous pulse schedule

with qx.PulseSchedule(labels) as seq2:
    seq2.call(seq1)
    seq2.barrier()
    seq2.add("Q00", qx.pulse.Blank(100))
    seq2.barrier()
    seq2.call(seq1.reversed())

seq2.plot()

In [ ]:
# Cross-resonance sequence

x180 = qx.pulse.Drag(duration=16, amplitude=0.02, beta=0.1)

zx90 = qx.pulse.CrossResonance(
    control_qubit="Q00",
    target_qubit="Q01",
    cr_amplitude=1.0,
    cr_duration=200,
    cr_ramptime=30,
    cr_phase=0.0,
    cancel_amplitude=0.01,
    cancel_phase=np.pi / 6,
    echo=True,
    pi_pulse=x180,
)

zx90.plot()

In [ ]:
# CNOT sequence

x90 = qx.pulse.Drag(duration=16, amplitude=0.01, beta=0.1)
y90 = x90.shifted(np.pi / 2)
z180 = qx.VirtualZ(np.pi)

hadamard = qx.PulseArray([z180, y90])

with qx.PulseSchedule(zx90.labels) as cnot:
    cnot.add("Q00", hadamard)
    cnot.add("Q01", hadamard)
    cnot.call(zx90)
    cnot.add("Q00", hadamard)
    cnot.add("Q01", hadamard)

cnot.plot()

In [ ]:
# CNOT sequence with CPMG

with qx.PulseSchedule(cnot.labels + ["Q02"]) as seq:
    seq.call(cnot)
    pi = qx.pulse.Drag(duration=16, amplitude=0.01, beta=0.1)
    tau = (cnot.duration - 2 * pi.duration) // 4
    seq.add("Q02", qx.pulse.CPMG(tau=tau, pi=pi))

seq.plot()